In [1]:
%%bash
chia version
cdv --version

1.4.0
cdv, version 1.0.8


# Tic Tac Toe

> Use [singleton_top_layer_v1_1](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer_v1_1.py)

Inner Puzzle is [tic tac toe coin](tic-tac-toe-coin.ipynb).

In [2]:
from pathlib import Path
import sys

# import singleton_helpers and tic_tac_toe
sys.path.insert(0, ".")
import tic_tac_toe
sys.path.insert(0, "..")
import singleton_helpers_v1_1

In [3]:
from chia.types.blockchain_format.program import Program
from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

tic_tac_toe_puzzle = tic_tac_toe.load_puzzle()
tic_tac_toe_coin_puzzle = tic_tac_toe.load_coin_puzzle()

In [4]:
from chia.wallet.puzzles import (singleton_top_layer_v1_1, p2_delegated_puzzle_or_hidden_puzzle)
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
bob: Wallet = network.make_wallet("bob")
await network.farm_block(farmer=alice)
await network.farm_block(farmer=bob)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')
print(f'bob balance:\t\t{bob.balance()}')
print(f'bob puzzle hash:\t{bob.puzzle_hash}')

alice balance:		2000000000000
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3
bob balance:		2000000000000
bob puzzle hash:	87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5


In [5]:
# prepare tic tac toe coin
# empty board
board = [0] * 9
# player one (x)
player = 1
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_puzzle(
    tic_tac_toe_puzzle, board, player
)
curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    alice.puzzle_hash, # player one (x)
    bob.puzzle_hash, # player two (o)
    curried_tic_tac_toe_puzzle)

# print(disassemble(curried_tic_tac_toe_puzzle))
# print()
# print(disassemble(curried_tic_tac_toe_coin_puzzle))

In [6]:
standard_txn_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
standard_txn_coin = standard_txn_coin_wrapper.as_coin()
standard_txn_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

inner_puzzle = curried_tic_tac_toe_coin_puzzle
standard_coin_message, coin_spends = singleton_helpers_v1_1.get_create_singleton_coin_spends(
                    standard_txn_coin,
                    standard_txn_puzzle,
                    odd_amount = 1023, 
                    inner_puzzle = inner_puzzle,
                    keys_values = [
                        ("game", "tic tac toe"),
                        ("player one", "alice"),
                        ("player two", "bob")
                    ]
              )
standard_txn_coin_spend = coin_spends[0]
launcher_coin_spend = coin_spends[1]
print("\nstarting coin spend:")
singleton_helpers_v1_1.print_json(standard_txn_coin_spend.to_json_dict())
print("\nlauncher coin spend:")
singleton_helpers_v1_1.print_json(launcher_coin_spend.to_json_dict())
print(f'\nlauncher id: {launcher_coin_spend.coin.name()}')
print(f'\nstandard_coin_message: {standard_coin_message.hex()}')

# print(disassemble(inner_puzzle))


starting coin spend:
{
    "coin": {
        "amount": 1750000000000,
        "parent_coin_info": "0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001",
        "puzzle_hash": "0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ff0bffff01ff02ffff03ffff09ff05ffff1dff0bffff1effff0bff0bffff02ff06ffff04ff02ffff04ff17ff8080808080808080ffff01ff02ff17ff2f80ffff01ff088080ff0180ffff01ff04ffff04ff04ffff04ff05ffff04ffff02ff06ffff04ff02ffff04ff17ff80808080ff80808080ffff02ff17ff2f808080ff0180ffff04ffff01ff32ff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff06ffff04ff02ffff04ff09ff80808080ffff02ff06ffff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080ffff04ffff01b0a042c855d234578415254b7870b711fb25e8f85beaa4a66bd0673d394c761fa156406c2e3bb375d5b18766d2a12cc918ff018080",
    "solution": "0xff80ffff01ffff33ffa0eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9ff8203ff80ffff3dffa0

In [7]:
# sign the standard txn spend
from blspy import AugSchemeMPL, PrivateKey
from chia.consensus.default_constants import DEFAULT_CONSTANTS

synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)

signature = AugSchemeMPL.sign(synthetic_sk,standard_coin_message)

# push txn to create an eve singleton
from chia.types.spend_bundle import SpendBundle

spend_bundle = SpendBundle(
    coin_spends,
    signature
)
await network.push_tx(spend_bundle)

{'additions': [Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>, amount=1023),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1749999998977),
  Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 7a56d20c59d68dd1b72c680d61aafef7d65d307c7b60a8f9c65da9c803ad3019>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: e3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001>, puzzle_hash=<bytes32: 4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3>, amount=1750000000000),
  Coin(parent_coin_info=<bytes32: 12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba>, puzzle_hash=<bytes32: eff07522495060c06

In [8]:
# spend eve singleton
network.sim.pass_blocks(1)
launcher_id = launcher_coin_spend.coin.name()
singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
            singleton_struct,
            inner_puzzle,
    )

singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id) 

lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(launcher_coin_spend)

odd_amount = 1023
position = 4
inner_solution = Program.to([
    odd_amount, 
    position
])

singleton_coin_spend = singleton_helpers_v1_1.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
    )

print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')
print(f'\nsingleton_puzzle:\n{singleton_puzzle.get_tree_hash()}')

print('\nsingleton_coin_spend:')
singleton_helpers_v1_1.print_json(singleton_coin_spend.to_json_dict())

singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
 'puzzle_hash': '0x7a56d20c59d68dd1b72c680d61aafef7d65d307c7b60a8f9c65da9c803ad3019'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}

inner_solution:
ff8203ffff0480

singleton_puzzle:
7a56d20c59d68dd1b72c680d61aafef7d65d307c7b60a8f9c65da9c803ad3019

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570",
        "puzzle_hash": "0x7a56d20c59d68dd1b72c680d61aafef7d65d307c7b60a8f9c65da9c803ad3019"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fff3480ffff01ff04ffff04ff20ffff04ff2fff808080ffff04ffff02ff3effff04ff02ffff04ff05ffff04ffff02ff2affff04ff02ffff04ff27ffff04ffff02ffff03ff77ffff01ff02ff36ffff04ff02ffff04ff09ffff04ff57ffff04ffff02ff2effff0

In [9]:
from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
singleton_helpers_v1_1.print_json(spend_bundle.to_json_dict())
await network.push_tx(spend_bundle)

{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_solutions": [
        {
            "coin": {
                "amount": 1023,
                "parent_coin_info": "0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570",
                "puzzle_hash": "0x7a56d20c59d68dd1b72c680d61aafef7d65d307c7b60a8f9c65da9c803ad3019"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fff3480ffff01ff04ffff04ff20ffff04ff2fff808080ffff04ffff02ff3effff04ff02ffff04ff05ffff04ffff02ff2affff04ff02ffff04ff27ffff04ffff02ffff03ff77ffff01ff02ff36ffff04ff02ffff04ff09ffff04ff57ffff04ffff02ff2effff04ff02ffff04ff05ff80808080ff808080808080ffff011d80ff0180ffff04ffff02ffff03ff77ffff0181b7ffff015780ff0180ff808080808080ffff04ff77ff808080808080ffff02ff3affff04ff02ffff04ff05ffff04ffff02ff0bf

{'additions': [Coin(parent_coin_info=<bytes32: 09626e520eb95437de86e181ae3e4d32663ac12cc515bc9d83d3c0e6f649be80>, puzzle_hash=<bytes32: f6dd76a3c69ad8e979949330ed244cf299fc07c468e0a693247ddcbcfe1efaf7>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, puzzle_hash=<bytes32: 7a56d20c59d68dd1b72c680d61aafef7d65d307c7b60a8f9c65da9c803ad3019>, amount=1023)]}

In [10]:
# get puzzle reveal and solution
singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(get_coin_records_by_parent_ids, launcher_id)
parent_id = singleton_coin.parent_coin_info
parent_coin_record = await get_coin_record_by_name(parent_id)
spent_block_index = parent_coin_record.spent_block_index
coin_spent = await get_puzzle_and_solution(parent_id, spent_block_index)
singleton_puzzle = coin_spent.puzzle_reveal.to_program()
singleton_solution = coin_spent.solution.to_program()
print(f'\nsingleton_coin:\n{singleton_coin}')
print(f'\nparent_coin_record:\n{parent_coin_record}')
# print(disassemble(singleton_puzzle))


singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x09626e520eb95437de86e181ae3e4d32663ac12cc515bc9d83d3c0e6f649be80',
 'puzzle_hash': '0xf6dd76a3c69ad8e979949330ed244cf299fc07c468e0a693247ddcbcfe1efaf7'}

parent_coin_record:
{'coin': {'amount': 1023,
          'parent_coin_info': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570',
          'puzzle_hash': '0x7a56d20c59d68dd1b72c680d61aafef7d65d307c7b60a8f9c65da9c803ad3019'},
 'coinbase': False,
 'confirmed_block_index': 3,
 'spent_block_index': 5,
 'timestamp': 1}


In [11]:
from clvm_tools.binutils import disassemble
# get last board and last player
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_puzzle_from_curried_coin_puzzle(curried_tic_tac_toe_coin_puzzle)
board_from_puzzle = tic_tac_toe.get_board_from_curried_puzzle(curried_tic_tac_toe_puzzle)
tic_tac_toe.print_board(board_from_puzzle)

player_from_puzzle = tic_tac_toe.get_player_from_curried_puzzle(curried_tic_tac_toe_puzzle)
print(f'player: {player_from_puzzle}')

# get position from spent singleton solution
# use it to get current board
print(disassemble(singleton_solution))
position_from_solution = tic_tac_toe.get_position_from_singleton_solution(singleton_solution)
print(f'position: {position_from_solution}')
_, current_board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position_from_solution)
tic_tac_toe.print_board(current_board)

   |   |   
---+---+---
   |   |   
---+---+---
   |   |   
player: 1
((0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba 1023) 1023 (1023 4))
position: 4
   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


In [12]:
result, current_board = await tic_tac_toe.get_current_state(get_coin_records_by_parent_ids, get_coin_record_by_name, get_puzzle_and_solution, launcher_id)
print(result)
tic_tac_toe.print_board(current_board)

0
   |   |   
---+---+---
   | x |   
---+---+---
   |   |   


In [13]:
# spend next singleton
network.sim.pass_blocks(1)
singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

# new curried_tic_tac_toe_coin_puzzle
player = 2
curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_puzzle(
    tic_tac_toe_puzzle, current_board, player
)

curried_tic_tac_toe_coin_puzzle = tic_tac_toe_coin_puzzle.curry(
    tic_tac_toe_coin_puzzle,
    alice.puzzle_hash, # player one (x)
    bob.puzzle_hash, # player two (o)
    curried_tic_tac_toe_puzzle)

inner_puzzle = curried_tic_tac_toe_coin_puzzle
singleton_puzzle: Program = singleton_top_layer_v1_1.puzzle_for_singleton(
    launcher_id,
    inner_puzzle
)

singleton_coin = await singleton_helpers_v1_1.get_unspent_singleton(
    get_coin_records_by_parent_ids, 
    launcher_id)

lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(singleton_coin_spend)

odd_amount = 1023
position = 1
inner_solution = Program.to([
    odd_amount,
    position
])

singleton_coin_spend = singleton_helpers_v1_1.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof, inner_solution
)

print(f'singleton_struct:\n{singleton_struct}')
print(f'singleton_coin:\n{singleton_coin}')
print(f'\nlineage_proof:\n{lineage_proof}')
print(f'\ninner_solution:\n{inner_solution}')

print('\nsingleton_coin_spend:')
singleton_helpers_v1_1.print_json(singleton_coin_spend.to_json_dict())

singleton_struct:
(<bytes32: 7faa3253bfddd1e0decb0906b2dc6247bbc4cf608f58345d173adb63e8b47c9f>, (<bytes32: 6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570>, <bytes32: eff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9>))
singleton_coin:
{'amount': 1023,
 'parent_coin_info': '0x09626e520eb95437de86e181ae3e4d32663ac12cc515bc9d83d3c0e6f649be80',
 'puzzle_hash': '0xf6dd76a3c69ad8e979949330ed244cf299fc07c468e0a693247ddcbcfe1efaf7'}

lineage_proof:
{'amount': 1023,
 'inner_puzzle_hash': '0x2950dbd372a6fe5d9e0fc2d2bcae3ba5339ed41df3ca45b94a8631bc5959c712',
 'parent_name': '0x6a4ba7e394f8d346deafcda74b26bcad649ed0cb691d7172b14970c4cf47a570'}

inner_solution:
ff8203ffff0180

singleton_coin_spend:
{
    "coin": {
        "amount": 1023,
        "parent_coin_info": "0x09626e520eb95437de86e181ae3e4d32663ac12cc515bc9d83d3c0e6f649be80",
        "puzzle_hash": "0xf6dd76a3c69ad8e979949330ed244cf299fc07c468e0a693247ddcbcfe1efaf7"
    },
    "puzzle_reveal": "0xff02ffff01f

In [14]:
from blspy import G2Element
spend_bundle = SpendBundle(
    [singleton_coin_spend],
    G2Element()
)
singleton_helpers_v1_1.print_json(spend_bundle.to_json_dict())
await network.push_tx(spend_bundle)

{
    "aggregated_signature": "0xc00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    "coin_solutions": [
        {
            "coin": {
                "amount": 1023,
                "parent_coin_info": "0x09626e520eb95437de86e181ae3e4d32663ac12cc515bc9d83d3c0e6f649be80",
                "puzzle_hash": "0xf6dd76a3c69ad8e979949330ed244cf299fc07c468e0a693247ddcbcfe1efaf7"
            },
            "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fff3480ffff01ff04ffff04ff20ffff04ff2fff808080ffff04ffff02ff3effff04ff02ffff04ff05ffff04ffff02ff2affff04ff02ffff04ff27ffff04ffff02ffff03ff77ffff01ff02ff36ffff04ff02ffff04ff09ffff04ff57ffff04ffff02ff2effff04ff02ffff04ff05ff80808080ff808080808080ffff011d80ff0180ffff04ffff02ffff03ff77ffff0181b7ffff015780ff0180ff808080808080ffff04ff77ff808080808080ffff02ff3affff04ff02ffff04ff05ffff04ffff02ff0bf

{'additions': [Coin(parent_coin_info=<bytes32: 26bc64e0f9e8a5dda42f898ce9b6d8b04cdfab16a9170134292fb214839955fc>, puzzle_hash=<bytes32: 5cea3f3d112e288d6667b01fdeff4f10cfc8c404de8c483e6528e02db2988946>, amount=1023)],
 'removals': [Coin(parent_coin_info=<bytes32: 09626e520eb95437de86e181ae3e4d32663ac12cc515bc9d83d3c0e6f649be80>, puzzle_hash=<bytes32: f6dd76a3c69ad8e979949330ed244cf299fc07c468e0a693247ddcbcfe1efaf7>, amount=1023)]}

In [15]:
result, current_board = await tic_tac_toe.get_current_state(get_coin_records_by_parent_ids, get_coin_record_by_name, get_puzzle_and_solution, launcher_id)
print(result)
tic_tac_toe.print_board(current_board)

0
   | o |   
---+---+---
   | x |   
---+---+---
   |   |   
